In [2]:
import tensorflow as tf
from tensorflow import keras



# Hyper parameter 튜닝
* 최적의 Hyper parameter 조합을 찾는 가장 좋은 방법은 모두 해보고 최적의 조합을 찾는 것이다.
* 이는 다시 말해 Hyperparameter 들로 이루어진 공간 , 차원에서 가장 최적의 parameter 조합을 찾아야 한다는 이야기 이다.
* 이러한 방법으로 parameter 조합들에 대해 cross validation을 적용할 수 있다.



In [9]:
def build_model(n_hidden=1, n_neurons=30 , learning_rate=3e-3, input_shape=[8]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
  model.add(keras.layers.Dense(1))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss="mse",optimizer=optimizer)
  return model

* 위의 코드를 통해 회귀를 위한 간단한 모델을 만들 수 있다.
* 즉 build_model 이라는 함수를 통해서 모델을 만들 수 있도록 하였다.

In [11]:
#KerasRegressor

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  This is separate from the ipykernel package so we can avoid doing imports until


* kerassRegressor 객체는 함수로 만들어진 모델을 감싸는 wrapper다.
* 이를 통해 아래 코드와 같게 scikit_learn에서 사용하는 것처럼 이 객체를 사용할 수 있다.

In [12]:
#캘리포니아 주택 가격 데이터 셋을 통해 예시를 살펴본다.
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full , X_test , y_train_full , y_test = train_test_split(
    housing.data, housing.target
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full , y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)
X_new = 

In [ ]:
keras_reg.fit(X_train, y_train , epochs=100,
              validation_data = (X_valid , y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test,y_test)
#y_pred = keras_reg.predict(X_new)

* 위의 코드에서 fit() 메소드에 전달한 값들은 모두 모델로 전달된다.
* cross validation set에서 최적의 모델을 찾아야 하므로 hidden layer 개수 , 뉴런 개수 , learning rate를 이용해 hyper parameter를 찾을 수 있다.

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden" : [0,1,2,3],
    "n_neurons" : np.arange(1, 100),
    "learning_rate" : reciprocal(3e-4,3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10 , cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

* 위의 코드를 통해 RandomizedSearch Cross Validation을 적용한다.
* 이 과정은 모델의 복잡도 , 데이터 셋의 크기 , cv 변수 , n_iter의 수에 따라 걸리는 시간이 달라진다.
* 이후 위의 코드는 랜덤서치 CV를 통해 찾은 최적의 Hyper parameter들로 이루어진 모델을 보여준다.

In [ ]:
rnd_search_cv.best_params_
#best parameters

In [ ]:
rnd_search_cv.best_score_
#best score 확인 ( 음수 mse 값으로 나옴 사이킷런은 loss가 아니라 score를 계산해서)

In [ ]:
model = rnd_search_cv.best_estimator_.model
#위와 같은 방식으로 베스트 모델을 가져울 수도 있다.

# Hyper parameter 탐색 라이브러리
* Hyperopt : 학습률 , 층개수 등에 대해 최적화를 수행하는 라이브러리
* Hypers , kopt , Talos : Keras 모델을 위한 parameter 최적화 라이브러리
* Keras tuner : 사용하기 쉬운 keras 최적화 라이브러리
* Scikit-Optimize(skopt) : 범용적으로 사용되는 최적화 라이브러리
* Spearmint : 베이즈 최적화 라이브러리
* Hyperband : Hyperband 논문 기반의 빠른 parameter 튜닝 라이브러리
* Sklearn-Deap : Grid서치CV와 비슷한 진화 알고리즘 기반의 최적화 라이브러리
* 이외에도 다양한 최적화 라이브러리 들이 존재한다.

# Hideen Layer
* 이론상 Hidden layer가 하나라도 뉴런의 개수가 많으면 복잡한 문제도 풀 수 있다.
* 하지만 deep 한 구조가 그렇지 않은 구조보다 parameter에 대한 효율성이 좋다.
* 즉 같은 문제를 해결한다고 생각할 떄 , deep한 구조가 그렇지 않은 구조에 비해 뉴런을 적게 사용하므로 더 높은 성능을 낼 수 있다는 것이다.
* 그래서 일반적으로 뉴런개수를 늘리는 것보다 층 수를 늘리는 것이 이득이 더 많다.

# 전이 학습 (Transfer Learning)
* 전이 학습은 어떠한 분야에서 학습된 신경망을 비슷하거나 새로운 분야에 사용되는 신경망의 학습에 사용하는 것을 말한다.
* 이러한 전이 학습을 위해서는 쌓여있는 네트워크 구조 즉 계층 구조가 필요하다.

# 뉴런의 개수
* 일반적으로 input과 output 뉴런의 개수는 문제 정의에 따라 결정된다.
* 일반적인 hidden layer 구성방식은 점점 내려갈 수록 각층의 뉴런 개수를 줄인다.
* 이는 대량의 저수준 특성들이 적은 수의 고수준 특성들로 합쳐질 수 있기 떄문이다.
* 뉴런의 개수도 층의 개수와 마찬가지로 점진적으로 늘려 나가다 Overfitting 되지 않도록 조기종료 혹은 규제를 사용하는 것이 간단하고 편하다.

# 학습률 (Learning Rate)
* 일반적으로 최적의 lr은 최대 lr의 절반 정도이다.
* 좋은 lr을 찾는 방법은 매우 낮은 lr부터 점진적으로 늘려나가 매우 큰 lr까지 반복하여 모델을 훈련하는 것이다.
* 이 과정을 통해 loss를 확인해 loss가 감소하다 다시 증가하는 부분을 찾을 수 있고 이를 통해 최적의 lr을 찾을 수 있다.
* lr은 다른 parameter들에 대해 의존적이다.특히 batch size에 영향을 많이 받는다.
* 따라서 다른 parameter를 수정했다면 , lr도 다시 튜닝해야 한다.

#optimizer
* 기본적인 경사 하강법 이외에도 다양한 optimizer들이 있다.
* 이러한 optimizer 선택도 영향이 있다.

# batch size
* batch 크기가 클수록 하드웨어 자원을 효율적으로 사용할 수 있다.
* 하지만 너무 큰 batch 크기를 사용하면 불안정한 결과를 낼 수도 있다.
* 큰 batch size는 시간을 크게 줄여주지만 결과가 불안정 할 수도 있다.
* 따라서 결과가 불안정 하다면 batch size를 조정하는 것도 고려해야 한다.

# activation function
* 다양한 activation function 이 있다.
* function들중 알맞은 function을 고르는 것도 중요하다.

# 반복 횟수
* 반복 횟수 역시 정할 수 있는 parameter다.
* 하지만 조기종료를 대부분 사용하므로 일반적인 경우에서는 튜닝할 필요가 없다.



